# Working with relational databases
## Create the engine

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Create engine
engine = create_engine('sqlite:///Chinook.sqlite')
# Save the table names to a list: table_names
table_names = engine.table_names()

# Print the table names
print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


/var/folders/4n/dll6nv094h5fm4m2lhzkmtd40000gn/T/ipykernel_45057/2987031341.py:4: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


## Create the connection

In [3]:
# Open engine connection: con
con = engine.connect()

# Query
rs = con.execute('SELECT * FROM Album')

# Save results of the query to DataFrame
df = pd.DataFrame(rs.fetchall())

# Close connection
con.close()

print(df.head())

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3


## Query and creating a dataframe using "with" (.fetchmany(n) or .fetchall())

In [4]:
# Perform query and save results to DataFrame
with engine.connect() as con:
    rs = con.execute('SELECT FirstName, Title, Address, City  FROM Employee')
    df = pd.DataFrame(rs.fetchmany(3))
    df.columns = rs.keys()

# Print the head of the DataFrame df
print(df.head())

  FirstName                Title              Address      City
0    Andrew      General Manager  11120 Jasper Ave NW  Edmonton
1     Nancy        Sales Manager         825 8 Ave SW   Calgary
2      Jane  Sales Support Agent        1111 6 Ave SW   Calgary


In [5]:
# Perform query and save results to DataFrame: df
with engine.connect() as con:
    rs = con.execute('SELECT * FROM Employee WHERE EmployeeId >= 6')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

# Print the head of the DataFrame df
print(df.head())

   EmployeeId  LastName FirstName       Title  ReportsTo            BirthDate  \
0           6  Mitchell   Michael  IT Manager          1  1973-07-01 00:00:00   
1           7      King    Robert    IT Staff          6  1970-05-29 00:00:00   
2           8  Callahan     Laura    IT Staff          6  1968-01-09 00:00:00   

              HireDate                      Address        City State Country  \
0  2003-10-17 00:00:00         5827 Bowness Road NW     Calgary    AB  Canada   
1  2004-01-02 00:00:00  590 Columbia Boulevard West  Lethbridge    AB  Canada   
2  2004-03-04 00:00:00                  923 7 ST NW  Lethbridge    AB  Canada   

  PostalCode              Phone                Fax                    Email  
0    T3B 0C5  +1 (403) 246-9887  +1 (403) 246-9899  michael@chinookcorp.com  
1    T1K 5N8  +1 (403) 456-9986  +1 (403) 456-8485   robert@chinookcorp.com  
2    T1H 1Y8  +1 (403) 467-3351  +1 (403) 467-8772    laura@chinookcorp.com  


In [6]:
# Open engine in context manager
with engine.connect() as con:
    rs = con.execute('SELECT * FROM Employee ORDER BY BirthDate')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df.head())

   EmployeeId  LastName FirstName                Title  ReportsTo  \
0           4      Park  Margaret  Sales Support Agent        2.0   
1           2   Edwards     Nancy        Sales Manager        1.0   
2           1     Adams    Andrew      General Manager        NaN   
3           5   Johnson     Steve  Sales Support Agent        2.0   
4           8  Callahan     Laura             IT Staff        6.0   

             BirthDate             HireDate              Address        City  \
0  1947-09-19 00:00:00  2003-05-03 00:00:00     683 10 Street SW     Calgary   
1  1958-12-08 00:00:00  2002-05-01 00:00:00         825 8 Ave SW     Calgary   
2  1962-02-18 00:00:00  2002-08-14 00:00:00  11120 Jasper Ave NW    Edmonton   
3  1965-03-03 00:00:00  2003-10-17 00:00:00         7727B 41 Ave     Calgary   
4  1968-01-09 00:00:00  2004-03-04 00:00:00          923 7 ST NW  Lethbridge   

  State Country PostalCode              Phone                Fax  \
0    AB  Canada    T2P 5G3  +1 (403)

In [7]:
with engine.connect() as con:
    rs = con.execute('SELECT Title, Name FROM Album INNER JOIN Artist ON Album.ArtistID = Artist.ArtistID')
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()

print(df.head())

                                   Title       Name
0  For Those About To Rock We Salute You      AC/DC
1                      Balls to the Wall     Accept
2                      Restless and Wild     Accept
3                      Let There Be Rock      AC/DC
4                               Big Ones  Aerosmith


## Simple SQL queries

In [8]:
# df = pd.read_sql_query("QUERY", engine)
df = pd.read_sql_query("SELECT * FROM Album", engine)

print(df.head())

   AlbumId                                  Title  ArtistId
0        1  For Those About To Rock We Salute You         1
1        2                      Balls to the Wall         2
2        3                      Restless and Wild         2
3        4                      Let There Be Rock         1
4        5                               Big Ones         3


In [9]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Album")
    df1 = pd.DataFrame(rs.fetchall())
    df1.columns = rs.keys()
# Both methods work equally
print(df.equals(df1))

True


In [10]:
df = pd.read_sql_query('SELECT * FROM Employee WHERE EmployeeId >= 6 ORDER BY BirthDate', engine)

# Print head of DataFrame
print(df.head())

   EmployeeId  LastName FirstName       Title  ReportsTo            BirthDate  \
0           8  Callahan     Laura    IT Staff          6  1968-01-09 00:00:00   
1           7      King    Robert    IT Staff          6  1970-05-29 00:00:00   
2           6  Mitchell   Michael  IT Manager          1  1973-07-01 00:00:00   

              HireDate                      Address        City State Country  \
0  2004-03-04 00:00:00                  923 7 ST NW  Lethbridge    AB  Canada   
1  2004-01-02 00:00:00  590 Columbia Boulevard West  Lethbridge    AB  Canada   
2  2003-10-17 00:00:00         5827 Bowness Road NW     Calgary    AB  Canada   

  PostalCode              Phone                Fax                    Email  
0    T1H 1Y8  +1 (403) 467-3351  +1 (403) 467-8772    laura@chinookcorp.com  
1    T1K 5N8  +1 (403) 456-9986  +1 (403) 456-8485   robert@chinookcorp.com  
2    T3B 0C5  +1 (403) 246-9887  +1 (403) 246-9899  michael@chinookcorp.com  


## Using sqlite3

In [11]:
import sqlite3

In [12]:
%load_ext sql
%sql sqlite:///Chinook.sqlite

In [13]:
%%sql
SELECT FirstName, Title, Address, City  
FROM Employee

 * sqlite:///Chinook.sqlite
Done.


FirstName,Title,Address,City
Andrew,General Manager,11120 Jasper Ave NW,Edmonton
Nancy,Sales Manager,825 8 Ave SW,Calgary
Jane,Sales Support Agent,1111 6 Ave SW,Calgary
Margaret,Sales Support Agent,683 10 Street SW,Calgary
Steve,Sales Support Agent,7727B 41 Ave,Calgary
Michael,IT Manager,5827 Bowness Road NW,Calgary
Robert,IT Staff,590 Columbia Boulevard West,Lethbridge
Laura,IT Staff,923 7 ST NW,Lethbridge


In [14]:
%%sql
SELECT Title, Name 
FROM Album 
    INNER JOIN Artist 
    ON Album.ArtistID = Artist.ArtistID

 * sqlite:///Chinook.sqlite
Done.


Title,Name
For Those About To Rock We Salute You,AC/DC
Balls to the Wall,Accept
Restless and Wild,Accept
Let There Be Rock,AC/DC
Big Ones,Aerosmith
Jagged Little Pill,Alanis Morissette
Facelift,Alice In Chains
Warner 25 Anos,Antonio Carlos Jobim
Plays Metallica By Four Cellos,Apocalyptica
Audioslave,Audioslave
